# Africa NDVI Animated Images

In [11]:
import ee
import geemap
import os
import glob

In [12]:
# Initialize google earth engine
ee.Initialize()

# Fetch a MODIS NDVI collection and select NDVI.
img_collection = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

# Define a mask to clip the NDVI data by.
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))

# Define the regional bounds of animation frames.
region = ee.Geometry.Polygon(
  [[[-18.698368046353494, 38.1446395611524],
    [-18.698368046353494, -36.16300755581617],
    [52.229366328646506, -36.16300755581617],
    [52.229366328646506, 38.1446395611524]]], None, False
)

In [13]:
# Add day-of-year (DOY) property to each image.
def clip_and_get_day_of_year(img):
    img = img.clip(mask)
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)

In [14]:
# Apply median reduction among matching DOY collections.
def match_day_of_year_and_reduce(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median())

In [15]:
img_collection = img_collection.map(clip_and_get_day_of_year)

# Choose dates:  min is 2000-02-18  max is 2021-12-19
year = 2000
date_start = f'{str(year)}-02-18'
date_end = f'{str(year)}-12-31'

# Get a collection of distinct images by 'doy'.
distinct_doy = img_collection.filterDate(date_start, date_end)

# Define a filter that identifies which images from the complete
# collection match the DOY from the distinct DOY collection.
filtered = ee.Filter.equals(leftField = 'doy', rightField = 'doy')

# Define a join.
joined = ee.Join.saveAll('doy_matches')

# Apply the join and convert the resulting FeatureCollection to an ImageCollection.
join_img_collection = ee.ImageCollection(joined.apply(distinct_doy, img_collection, filtered))

img_composite = join_img_collection.map(match_day_of_year_and_reduce)

# Define visualization parameters.
vis_params = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 5,
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    "#000004FF", "#0B0724FF", "#210C4AFF", "#3D0965FF", "#56106EFF", "#71196EFF", 
    "#89226AFF", "#A32C61FF", "#BB3754FF", "#D14545FF", "#E35932FF", "#F1721EFF",
    "#F98C0AFF", "#FCAA0FFF", "#F9C932FF", "#F2E865FF", "#FCFFA4FF"
  ]
}

In [16]:
temp_file_name = f"animations/temp_ndvi_{str(year)}.gif"
file_name = f"animations/ndvi_{str(year)}.gif"
index_array = img_composite.aggregate_array('system:index').getInfo()

In [17]:
geemap.download_ee_video(img_composite, vis_params, temp_file_name)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\Cullen\OneDrive\Documents\CropMosaiks\NDVI_Images\animations\temp_ndvi_2000.gif


In [18]:
geemap.add_text_to_gif(
    in_gif = temp_file_name, 
    out_gif = file_name,
    xy= ('75%', '1%'),
    text_sequence=index_array,
    font_type='arial.ttf',
    font_size=20,
    font_color='white',
    add_progress_bar=True,
    progress_bar_color='white',
    progress_bar_height=5,
    duration=200, # milliseconds per frame so 200 is 5 fps, 100 is 10 fps etc
    loop=0
)

In [19]:
temp_list = glob.glob("animations/temp*.gif")
for file in temp_list:
  os.remove(file)